In [1]:
import re

Для всех задач, где формулировка вида "найдите в тексте", тексты для проверки вы можете выбирать абсолютно любые. 

#### Задача 1. 

Найдите в тексте все смайлики вида :(, :), :-(, :-).

In [20]:
text = "Любой :( текст со :-) смайликами :( :-) :^( :^)"
for m in re.finditer(r":[-^]*[()]", text):
    print(m.group())

:(
:-)
:(
:-)
:^(
:^)


#### Задача 2.

Некоторые любят выделять особо важные места в тексте с помощью нижнего подчёркивания, например: "...здесь \_очень важно\_ отметить..." (дело в том, что нижние подчеркивания в некоторых редакторах автоматически превращаются в курсив).

Ищем такие выделения, но не длиннее 5 слов.

In [4]:
# your code here
text = '_Некоторые любят выделять особо важные места_ в тексте с помощью нижнего подчёркивания, например: "...здесь _очень важно_ отметить..." (_дело в том_, что _нижние подчеркивания_ в некоторых _редакторах_ автоматически превращаются в курсив).'

# Я не использую \w, потому что он содержит _, и из-за того в тексте выше
# регулярка находит "_нижние подчеркивания_ в некоторых _"
# вместо "_нижние подчеркивания_" и "_редакторах_"
for m in re.finditer(r"_(?:[A-Za-zА-Яа-яёË]+\s?){1,5}?_", text):
    print(m.group())

_очень важно_
_дело в том_
_нижние подчеркивания_
_редакторах_


#### Задача 3.

Найдите в тексте все куски, которые взяты в круглые скобки. 

In [26]:
text = "текст с (круглыми скобками), но без вложенных (круглых скобок)"
text2 = "это обычный текст, (взятый в (круглые) скобки, ничего (необычного)) :))"

# если нет вложенных, то
for m in re.finditer(r"\(.+?\)", text):
    print(m.group())

# если есть, то я не знаю, есть такой набросок
# работает в regex101, но даже так не ищет вложенные, и если на конце больше 1 ), то включает их все
for m in re.finditer(r"(\(.*\1*.*\))", text2):
    print(m.group())

(круглыми скобками)
(круглых скобок)


error: cannot refer to an open group at position 5

#### Задача 4.

С помощью os.listdir() обходим все файлы с музыкой в папке; при этом в папке, которую мы обходим, нужные нам файлы называются в таком формате:

    имя исполнителя - номер песни - название песни.mp3
    
(при этом дефисы могут попасться как в имени исполнителя, так и в названии песни тоже). Программа должна:

1. Обрабатывать только файлы, которые удовлетворяют этому шаблону (открывать их не будем, давайте будем просто сообщать, что X - это песенка исполнителя Y)
2. По итогам работы собрать статистику с перечнем исполнителей и их песен в виде:

        имя исполнителя:
            номер песни. название песни
        имя исполнителя:
            номер песни. название песни

In [95]:
import os

singers: dict[str, dict[str, int]] = {}

for d in os.listdir("fake_mp3s"):
    mp3 = re.search(r"mp3", d)
    if not mp3:
        continue
    m = re.search(r".+(?= - \d)", d)
    if m:
        if m.group() not in singers:
           singers[m.group()] = {}

        i = re.search(r"(?<= - )\d+(?= - )", d)
        if i:
            n = re.search(r"(?<=\d - ).+?(?=\.mp3)", d)
            if n:
                singers[m.group()][n.group()] = i.group()

for (singer, songs) in singers.items():
    print(f"{singer}:")
    for (name, index) in songs.items():
        print(f"\t{index}. {name}")


another singer:
	2. song 2
	3. song 3
	1. song 1
new-singer:
	2. song-2
	1. song-1
singer:
	1. song 1
	2. song 2


#### Задача 5. 

Найдите в тексте смайлики вида \*\_\*, \^\_\^, T\_T (а смайлики вида o\_O нам не нужны).

In [18]:
text = "*_*, ^_^, T_T o_O ._."

for m in re.finditer(r'(.)_\1', text):
    print(m.group())

*_*
^_^
T_T
._.


#### Задача 6. 

Пишем программу - помощник обнаружения тавтологии: ищем в тексте повторяющиеся слова, которые находятся на расстоянии от 2 до 10 слов друг от друга. В простом случае считаем, что пунктуации у нас нет, в сложном можете попробовать ее тоже учесть. 

In [17]:
# your code here
text = "текст, хороший текст, текст, где есть тавтология, никто не любит тавтологию, надоела эта тавтология, надоела. один, два, три, четыре, пять, шесть, семь, восемь, девять, десять. тавтология"

# \b(\w{2,})\w{0,3}[\b,.!:; ]+ — ищем слово, его границы — \b и пунктуация, корень слова в группе () 1, за группой — возможное окончание
# (?=(?:\w+[\b,. !:;]+){2,10}\1) — lookahead, ищем от 2 до 10 слов, после которых \1 (группа 1)
for m in re.finditer(r"\b(\w{2,})\w{0,3}[\b,.!:; ]+(?=(?:\w+[\b,. !:;]+){2,10}\1)", text):
    print(m.group())

текст, 
тавтология, 
тавтологию, 
надоела 


#### Задача 7. 

Представим, что есть некоторые данные, представляющие из себя последовательность символов. С помощью регулярного выражения надо найти в них подпоследовательности, являющимися палиндромами, длины не менее 6-ти и не более 11 символов. (Подумайте, а хорошо ли подходят регулярные выражения для решения этой задачи?)

In [22]:
# Подумайте, а хорошо ли подходят регулярные выражения для решения этой задачи?
# После домашек кажется, что регулярные выражения в приципе не для многих задач подходят
# Ну а вообще, если мы знаем, что палиндром длиной 6-11, то проблем нет, а если он не заданной длины, то задача становится очень сложной,
# Потому что в re нет рекурсий

text = "11\n\
121\n\
1221\n\
12321\n\
123321\n\
1234321\n\
12344321\n\
123454321\n\
1234554321\n\
12345654321\n"
for m in re.finditer(r"(.)(.)(.)(.)?(.)?.?\5?\4?\3\2\1", text):
    print(m.group())

123321
1234321
12344321
123454321
1234554321
12345654321


#### Задача 8.

Найти в тексте все употребления слов "понравилось" и "впечатлило" такие, что непосредственно перед ними НЕ стоит "не". Время прошедшее, но род/число могут быть любыми.

In [17]:
# your code here
text = "мне понравилось, меня впечатлило, мне не понравилось, меня не впечатлило, мне понравилась игра, меня впечатлил его поступок, она меня впечатлила, он мне понравился, они меня впечатлили, они меня не впечатлили, они мне не понравились, они мне понравились"

for m in re.finditer(r"(?<! [Нн][Ее] )(?:понравил(?:ось|ась|ся|ись)|впечатлил(?:а|о|и)?)", text):
    print(m.group())

понравилось
впечатлило
понравилась
впечатлил
впечатлила
понравился
впечатлили
понравились


#### Задача 9. 

Дан текст, каждая строка является именем некоторого файла (без директории). Напишите регулярное выражение, которое захватывает имена файлов без расширения, при этом интерес представляют только те файлы, у которых расширение не .bat и не .txt.

In [125]:
with open("files.txt", "r") as f:
    for line in f.readlines():
        m = re.search(r"(.+)\.(?!txt|bat)", line)
        if m:
            print(m.group(1))

hw06
hw06
ad


#### Задача 10.

Дан текст, каждая строка которого является полным или относительным путём к некоторому файлу.

Напишите регулярное выражение, которое захватывает:
1. директорию, в которой лежит файл;
2. только имя файла (без расширения);
3. только расширение;
При этом:
- нужны только файлы, у которых расширение не .bat и не .txt.
- пути могут быть как в Unix, так и в Windows формате (https://ru.wikipedia.org/wiki/Путь_к_файлу).
- расширение, если оно есть, начинается с точки. Файлы могут быть без расширения вовсе (в этом случае на месте расширения должно стоять None или "")
- скрытые файлы могут начинаться с точки (например, .bashrc - и это не расширение)
- относительный путь может содержать только название файла, в этом случае вместо директории выведите None или ""
- в остальных случаях директория должна заканчиаться на разделитель директорий. Наприемр, в Unix-системах - "/" - это путь к корневой директории.

Требуется получить список из кортежей, каждый кортеж содержит извлечённые данные.

(Расширение в целом может содержать всё, что угодно, но разделителей директорий не может быть в именах файлах и расширениях. https://en.wikipedia.org/wiki/List_of_filename_extensions )

In [49]:
def get_paths(path):
    lst = []
    with open(path) as f:
        for line in f.readlines():
            if re.search(r"(.bat\n|.txt\n)", line):
                continue
            dir = re.search(r".+[/\\](?!.*[\\/])", line)
            if not dir:
                dir = None
            else:
                dir = dir.group()
            name = re.search(r"[^/\\]+(?=\..*$)", line)
            if not name:
                name = None
            else:
                name = name.group() 
            ext = re.search(r"\.[^\.\n]+(?=$)", line)
            if not ext:
                ext = None
            else:
                ext = ext.group()
            lst.append((dir, name, ext))
    return lst

get_paths("dirs.txt")
        

[('uni_homework/HW04/', 'hw04', '.py'),
 (None, 'hw04', '.py'),
 ('~/git/uni_homework/HW04/.jukit/', 'hw04_outhist', '.json'),
 ('~/git/uni_homework/HW03/', 'hw03', '.ipynb'),
 ('uni_homework/HW03/', 'hw03', '.py'),
 ('/Users/egor/uni_homework/HW06/', 'hw06', '.ipynb'),
 ('~/git/uni_homework/HW06/', 'hw06', '.py'),
 ('~/git/uni_homework/HW06/fake_mp3s/', 'another singer - 2 - song 2', '.mp3'),
 ('uni_homework/HW06/fake_mp3s/', 'another singer - 3 - song 3', '.mp3'),
 ('/Users/egor/uni_homework/HW06/fake_mp3s/', 'not_a_song', '.mp4'),
 ('uni_homework/HW06/fake_mp3s/', 'new-singer - 2 - song-2', '.mp3'),
 (None, 'new-singer - 2 - song-2', '.mp3'),
 ('/Users/egor/uni_homework/HW06/fake_mp3s/', 'singer - 1 - song 1', '.mp3'),
 ('uni_homework/HW06/fake_mp3s/', 'singer - 2 - song 2', '.mp3'),
 ('uni_homework/HW06/fake_mp3s/', 'new-singer - 1 - song-1', '.mp3'),
 ('uni_homework/.jukit/', 'hw06_outhist', '.json'),
 ('/Users/egor/uni_homework/HW06/fake_mp3s/', 'author - 1 - video', '.mp4'),
 ('

#### Задача 11.

Увеличить все встретившиеся в тексте четные числа на 1, а нечетные на 2, в том числе и отрицательные. Перед числом может стоять знак. Считаем, что между знаком и числом пробела нет.

In [6]:
# your code here
text = "1 банан, 2 банана, 3 банана, -1 банан, 101 банан, -102 банана, -10 бананов"
print(re.findall(r"-?\d+", text))
text = re.sub(r"-?\d+", lambda m: str(int(m.group()) + 1), text)
text

['1', '2', '3', '-1', '101', '-102', '-10']


'2 банан, 3 банана, 4 банана, 0 банан, 102 банан, -101 банана, -9 бананов'

#### Задача 12.

1. Текст состоит из строк (lines). Удалите одним регулярным выражением все пробелы в началах всех строк, и все пробелы в концах всех строк.
2. То же, но теперь не только обычные пробелы, а все пробельные символы. При этом, дополнительно нужно удалить все пустые строки (пустые вообще, а так же состоящие только из пробельных символов). Попробуйте сделать это за одну операцию замены.
3. Сделайте то же самое без регулярных выражений в одну строчку, используя bilt-in функцию filter.

In [53]:
# 1
text = []
with open("text.txt", "r") as f:
    for line in f.readlines():   
        text.append(re.sub(r"(^ *| *$)", "", line))

with open("text1.txt", "w") as f:
    for line in text:
        f.write(line)


In [62]:
# 2
text = []
with open("text.txt", "r") as f:
    for line in f.readlines():   
        text.append(re.sub(r"(^\s*|[ \f\r\t\v]*$)", "", line))

with open("text2.txt", "w") as f:
    for line in text:
        f.write(line)


In [61]:
# 3
with open("text.txt", "r") as f:
    # for line in f.readlines():
    #     line = list(filter(lambda x: x not in ["\n", "\t", "\r", " ", "\f", "\v"], line))
    #     a = ''.join(line)
    #     print(a)

    # Немного читерства с использванием map(), зато в одну строчку и есть фильтер.
    # Просто не могу придумать, возможно ли передать в фильтр функцию, которая бы смогла удалить только начальные и конечные пробелы
    print("".join(list(filter(lambda l: l.strip() != "", map(lambda l: l.strip() + "\n", f.readlines())))))
    



Это предложение не содержит пробелов вначале и в конце
Это же содержит в конце
А это в начале
Это вообще начинается с таба,   и кончается им же
Выше пустая строка с табами и пробелами
Выше пустая строка без ничего

